In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from openpyxl import load_workbook

In [11]:
doEverything()

Enter the first year from which you want data: 2005


###### Phantom browser options

In [3]:
phantomOptions = webdriver.ChromeOptions()
phantomOptions.add_argument('--headless')
# phantomOptions.add_argument('--no-proxy-server')
# phantomOptions.add_argument("--proxy-server='direct://'")
# phantomOptions.add_argument("--proxy-bypass-list=*")
# phantomOptions.add_argument("--log-level=3")

### Collect all data (Per Game Stats) from wanted years, add it to a dictionary with year, dataframe as key,value set.

In [4]:
def getData(dtype, yearsWanted):
    
    if dtype == "team":
        
        url = "https://www.basketball-reference.com/leagues/NBA_{}.html"
        
        table_id = ['team-stats-per_game','team-stats-per_poss', 'opponent-stats-per_game',
                    'opponent-stats-per_poss','misc_stats']

        
    if dtype == "player":
        url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
        
        table_id = ['per_game_stats']
        
       
    dfs = []
    
    for year in yearsWanted:

        # set driver type
        #driver = webdriver.Chrome(options=phantomOptions)
        driver = webdriver.Chrome()

        
        driver.get(url.format(year))

        HTML = driver.execute_script("return document.documentElement.outerHTML")

        driver.quit()

        soup = BeautifulSoup(HTML)
        
        dfLeft = pd.DataFrame()
        
        # for every data table that we don't have data for in the year of this iteration
        idx = 0
        
        for table in table_id:

            dataTable = soup.find('table', {'id':table})

            # use findALL() to get the column headers
            if table == "misc_stats":
                labs = 1
                limit = 2
            else:
                labs = 0
                limit = 1
                
            dataTable.findAll('tr', limit=limit)[labs]

            # use getText()to extract the text we need into a list
            headers = [th.getText() for th in dataTable.findAll('tr', limit=limit)[labs].findAll('th')]

            # exclude the first column as we will not need the ranking order for the analysis
            if table != "misc_stats":
                headers[9] = '3P%'
                headers[12] = '2P%'
            
            headers = headers[1:]
         
            rows = dataTable.findAll('tr', class_=lambda x: x != 'thead' or 'average_table no_ranker')[1:]
            
            stats = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(len(rows))]
                 
            dfRight = pd.DataFrame(stats, columns = headers)
                   
            if idx == 0:
                dfLeft = dfRight
                idx += 1
            
            else:
                if dtype == "team":
                    #dfRight = dfRight.sort_values(by=['Team'])
                    dfLeft = pd.merge(dfLeft, dfRight, on='Team')


                if dtype == "player":
                    #dfRight = dfRight.sort_values(by=['Player'])
                    dfLeft = pd.merge(dfLeft, dfRight, on='Player')
            
                         
            # add year to the second column of the data frame for each year       
            if dfLeft.columns[1] != "Year":
                dfLeft.insert(loc=1, column="Year", value=year)
                
            
        dfLeft = dfLeft.dropna(thresh=len(list(dfLeft.columns))-2)
        
        dfs.append(dfLeft)    
           
    dfFinal = pd.concat(dfs, axis=0, ignore_index=True)

    return dfFinal

## Outcome Data

#### Find winner of game

In [5]:
def winner(dataframe):
    if (dataframe['VisitorPTS'] > dataframe['HomePTS']):
        return dataframe['Visitor/Neutral']
    else:
        return dataframe['Home/Neutral']

#### Get outcome data for given years

In [6]:
def getOutcomeData(yearsWanted):
    
    
    url = "https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html"
    
    months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
    
    dfs = []
    
    for year in yearsWanted:
        
        for month in months:
            
            if year == 2020 and month == "april":
                break
#             key = str(year) + "_" + str(month)
            
#             # check to see if the data is already in the dictionary
#             if key in outcomeData:
#                 continue

            # set driver type
            #driver = webdriver.Chrome(options=phantomOptions)
            driver = webdriver.Chrome()


            driver.get(url.format(year, month))

            HTML = driver.execute_script("return document.documentElement.outerHTML")
            
            if 'Page Not Found (404 error)' in driver.page_source:
                driver.quit()
                continue
                
            driver.quit()

            soup = BeautifulSoup(HTML)

            dataTable = soup.find('table', {'id':'schedule'})

            # use findALL() to get the column headers
            dataTable.findAll('tr', limit=1)[0]

            # use getText()to extract the text we need into a list
            headers = [th.getText() for th in dataTable.findAll('tr', limit=1)[0].findAll('th')]

            # exclude the first column as we will not need the ranking order for the analysis
            headers = headers[1:]

            # avoid all header rows
            rows = dataTable.findAll('tr', class_=lambda x: x != 'thead')[1:]
                            
            stats = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(len(rows))]
        
            
            outcomeDF = pd.DataFrame(stats, columns = headers)
            
            # reformat dataframes to only include teams, points, attendence, and column for who won
            outcomeDF = outcomeDF.drop(['Start (ET)', 'Notes', '\xa0'], axis=1)
            outcomeDF.columns = ['Visitor/Neutral', 'VisitorPTS', 'Home/Neutral', 'HomePTS', 'Attend.']
            
            # convert integer columns to integers
            outcomeDF['VisitorPTS'] = outcomeDF['VisitorPTS'].astype(str).astype(int)
            outcomeDF['HomePTS'] = outcomeDF['HomePTS'].astype(str).astype(int)
            outcomeDF['Attend.'] = outcomeDF['Attend.'].replace(',','', regex=True)
            outcomeDF['Attend.'] = outcomeDF['Attend.'].astype(str).astype(int)
            
            
            # make any changes or calculations to the data set below here
            
            # create column for total points scored
            outcomeDF['Total Points'] = outcomeDF['VisitorPTS'] + outcomeDF['HomePTS']
            
            # find the winner of the game
            outcomeDF['Winner'] = outcomeDF.apply(winner, axis=1)
            
            #record the month and year of the game
            outcomeDF.insert(loc=0, column="Year", value=year)
            outcomeDF.insert(loc=1, column="Month", value=month)  
            
            dfs.append(outcomeDF)
            
            
    #dfFinal = pd.concat(dfs)
    dfFinal = pd.concat(dfs, axis=0, ignore_index=True)

    
    return dfFinal

In [7]:
def dataRange():
    startYear = int(input("Enter the first year from which you want data: "))
    currentYear = 2020
    yearsWanted = []

    for i in range(currentYear-startYear+1):
        yearsWanted.append(startYear+i)
        
    return yearsWanted

In [8]:
def collectData(yearsWanted):
    teamData = getData('team', yearsWanted)
    playerData = getData('player', yearsWanted)
    outcomeData = getOutcomeData(yearsWanted)
    
    return[teamData, playerData, outcomeData]

## Export data to Excel spreadsheet

In [9]:
def writeToExcel(teamDF, playerAnnualDF, outcomeDF):

    path = r"/Users/justinholmes/Desktop/Sports_Model/Data/NBA_data.xlsx"

    with pd.ExcelWriter(path, engine='xlsxwriter') as writer: 

        teamDF.to_excel(writer, "Team Data", index=False)
        playerAnnualDF.to_excel(writer, "Player Data", index=False)
        outcomeDF.to_excel(writer, "Outcome Data", index=False)

        writer.save() 
        writer.close()

In [10]:
def doEverything():
    
    years = dataRange()
    
    allDF = collectData(years)
    
    teamDF = allDF[0]
    playerAnnualDF = allDF[1]
    outcomeDF = allDF[2]
    
    writeToExcel(teamDF, playerAnnualDF, outcomeDF)